In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pylab as plt
import datetime
import locale
locale.setlocale(locale.LC_NUMERIC, "")
import subprocess
from stock_util.data_processor import DataLoader
from stock_util.model import Model


from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Conv1D, MaxPooling1D, AtrousConvolution1D, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.wrappers import Bidirectional
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.initializers import *
from keras.backend import clear_session

Using TensorFlow backend.


In [2]:
BID_QUERY='index="forex" sourcetype="stock:quote:eur_usd" NOT date_wday="saturday" NOT date_wday="sunday" earliest="1/28/2019:00:00:00" latest=-1mon|where date_hour>=0 AND date_hour<17  | eval _time=strftime(_time,"%m-%d-%Y-%H:%M:%S") | eval bid.v=volume | eval ask.v=volume| table _time bid.o,bid.h,bid.l,bid.c,bid.v,date_wday,date_hour,date_minute,date_second | sort 0 _time'
ASK_QUERY='index="forex" sourcetype="stock:quote:eur_usd" NOT date_wday="saturday" NOT date_wday="sunday" earliest="1/28/2019:00:00:00" latest=-1mon| where date_hour>=0 AND date_hour<17  |eval _time=strftime(_time,"%m-%d-%Y-%H:%M:%S") | eval bid.v=volume | eval ask.v=volume| table _time ask.o,ask.h,ask.l,ask.c,ask.v,date_wday,date_hour,date_minute,date_second | sort 0 _time'

In [3]:
loader=DataLoader()

In [4]:
def get_data():
    global loader
    loader.set_queries(BID_QUERY,ASK_QUERY)
    return loader.get_days(50000)

In [5]:
df=get_data()
print(df.head())
print(df.tail())

              Datetime   Open_x   High_x    Low_x  Close_x  Volume_x  \
0  01-28-2019-00:00:00  1.14199  1.14202  1.14192  1.14192    474.40   
1  01-28-2019-00:01:00  1.14192  1.14193  1.14178  1.14185    453.21   
2  01-28-2019-00:02:00  1.14185  1.14191  1.14182  1.14190    282.03   
3  01-28-2019-00:03:00  1.14190  1.14193  1.14183  1.14193    271.06   
4  01-28-2019-00:04:00  1.14193  1.14193  1.14192  1.14192    184.59   

  Weekday_x   Open_y   High_y    Low_y  Close_y  Volume_y Weekday_y  
0    monday  1.14189  1.14189  1.14174  1.14180    474.40    monday  
1    monday  1.14183  1.14187  1.14178  1.14186    453.21    monday  
2    monday  1.14187  1.14189  1.14180  1.14187    282.03    monday  
3    monday  1.14188  1.14190  1.14188  1.14190    271.06    monday  
4    monday  1.14187  1.14187  1.14179  1.14184    184.59    monday  
                  Datetime   Open_x   High_x    Low_x  Close_x  Volume_x  \
49995  04-05-2019-01:04:00  1.12262  1.12262  1.12259  1.12260    191.9

In [6]:
def lr_test():
    loader.window=90
    loader.forecast=45
    X_train, X_test, y_train, y_test = loader.get_train_test(df,0.9)
    
    #for bsize in [50,100,150]:
    for bsize in [100,150]:
        for lr in [0.001,0.002,0.003]:
            for rfactor in [0.8,0.85,0.9,0.95]:
                for patience in [5,10,15,20,25,30,35,40,45,50]:
                    for window in [90]:
                        for forecast in [45]:        
                            model=Model(loader.window,10,"test.h5","test_save.h5",loader.forecast,100)
                            model.set_logging(1)
                                                                    
                            # model parameters
                            model.batch_size=bsize
                            model.lr_reduce_factor=rfactor
                            model.patience=patience
                            model.lr=lr
                            model.test_name="Learn Rate - w:90 f:45 e:100"
                            model.test_desc="lr:"+str(lr)+"-bsize:"+str(bsize)+"-rfactor:"+str(rfactor)+"-patience:"+str(patience)
                            
                            model.build_model()
                            model.train(X_train, y_train,X_test, y_test)

def step_test():
    loader.window=90
    loader.forecast=45
    for step in [8]:
        model=Model(loader.window,10,"test.h5","test_save.h5",loader.forecast,20)
        model.set_logging(1)

        # model parameters
        model.batch_size=50
        model.lr_reduce_factor=0.8
        model.patience=20
        model.lr=0.001
        model.build_model()
        model.test_name="Steps Value - w:90 f:45 e:100"
        model.build_model() 
        
        model.test_desc="lr:"+str(0.001)+"-bsize:"+str(50)+"-rfactor:"+str(0.8)+"-patience:"+str(20)+"-steps:"+str(step)+" "
        X_train, X_test, y_train, y_test = loader.get_train_test(df,0.9,step)


        model.train(X_train, y_train,X_test, y_test)

def window_forecast_test():
#    for window in [10,20,30,40,50,60,70,80]:
#        for forecast in range(5,120):
    for window in [50,60,70,80]:
        for forecast in range(5,120):
            loader.window=window
            loader.forecast=forecast
            save_file="models/forex_"+str(window)+"_"+str(forecast)+".h5"
            model=Model(loader.window,10,"forex.h5",save_file,loader.forecast,30)
            model.set_logging(1)

            # model parameters
            model.batch_size=50
            model.lr_reduce_factor=0.8
            model.patience=20
            model.lr=0.001
            step=8
            model.test_name="Window/Forecast Models 1 - lr:"+str(0.001)+"-bsize:"+str(50)+"-rfactor:"+str(0.8)+"-patience:"+str(20)+"-steps:"+str(step)
            model.build_model()
            
            model.test_desc="window:"+str(window)+" forecast:"+str(forecast)
            X_train, X_test, y_train, y_test = loader.get_train_test(df,0.9,step)


            model.train(X_train, y_train,X_test, y_test)
            
    
def singletest():
    loader.window=60
    loader.forecast=90
    model=Model(loader.window,10,"test.h5","forex.h5",loader.forecast,50)
    model.set_logging(1)

    # model parameters
    model.batch_size=50
    model.lr_reduce_factor=0.8
    model.patience=20
    model.lr=0.001
    step=8
    model.test_name="Single - lr:"+str(0.001)+"-bsize:"+str(50)+"-rfactor:"+str(0.8)+"-patience:"+str(20)+"-steps:"+str(step)
    model.build_model()
            
    model.test_desc="window:"+str(loader.window)+" forecast:"+str(loader.forecast)
    X_train, X_test, y_train, y_test = loader.get_train_test(df,0.9,step)


    model.train(X_train, y_train,X_test, y_test)

In [7]:
#singletest()
#lr_test() # w90, f45 bsize50, lr:001 patience: 20
#step_test() #8 best
window_forecast_test() #~ w:60, f:90 best

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Doing ternary tensor with time length of  50000  and  8 steps
0 windows sized
500 windows sized
1000 windows sized
1500 windows sized
2000 windows sized
2500 windows sized
3000 windows sized
3500 windows sized
4000 windows sized
4500 windows sized
5000 windows sized
5500 windows sized
Tensor X/Y len:  5907  /  5907

Testing dataset dimension: [ 591 / 5316 ]

Instructions for updating:
Use tf.cast instead.
Train on 5316 samples, validate on 591 samples
Epoch 1/30
5316/5316 [==============================] - 1s 258us/step - loss: 1.1803 - acc: 0.3811 - val_loss: 1.0523 - val_acc: 0.4078

Epoch 00001: val_loss improved from inf to 1.05231, saving model to models/forex_50_5.h5
Epoch 2/30
5316/5316 [==============================] - 1s 100us/step - loss: 1.0991 - acc: 0.4048 - val_loss: 1.0581 - val_acc: 0.

5310/5310 [==============================] - 1s 96us/step - loss: 1.0606 - acc: 0.4226 - val_loss: 1.0437 - val_acc: 0.4220

Epoch 00004: val_loss did not improve from 1.04231
Epoch 5/30
5310/5310 [==============================] - 0s 92us/step - loss: 1.0553 - acc: 0.4235 - val_loss: 1.0447 - val_acc: 0.4203

Epoch 00005: val_loss did not improve from 1.04231
Epoch 6/30
5310/5310 [==============================] - 0s 92us/step - loss: 1.0505 - acc: 0.4190 - val_loss: 1.0447 - val_acc: 0.4271

Epoch 00006: val_loss did not improve from 1.04231
Epoch 7/30
5310/5310 [==============================] - 0s 93us/step - loss: 1.0410 - acc: 0.4296 - val_loss: 1.0470 - val_acc: 0.4237

Epoch 00007: val_loss did not improve from 1.04231
Epoch 8/30
5310/5310 [==============================] - 1s 97us/step - loss: 1.0383 - acc: 0.4294 - val_loss: 1.0456 - val_acc: 0.4271

Epoch 00008: val_loss did not improve from 1.04231
Epoch 9/30
5310/5310 [==============================] - 1s 95us/step - loss:


Epoch 00013: val_loss did not improve from 1.03113
Epoch 14/30
5304/5304 [==============================] - 1s 110us/step - loss: 1.0299 - acc: 0.4334 - val_loss: 1.0316 - val_acc: 0.4407

Epoch 00014: val_loss did not improve from 1.03113
Epoch 15/30
5304/5304 [==============================] - 0s 88us/step - loss: 1.0233 - acc: 0.4436 - val_loss: 1.0315 - val_acc: 0.4407

Epoch 00015: val_loss did not improve from 1.03113
Epoch 16/30
5304/5304 [==============================] - 0s 91us/step - loss: 1.0291 - acc: 0.4383 - val_loss: 1.0314 - val_acc: 0.4373

Epoch 00016: val_loss did not improve from 1.03113
Epoch 17/30
5304/5304 [==============================] - 0s 93us/step - loss: 1.0251 - acc: 0.4359 - val_loss: 1.0314 - val_acc: 0.4407

Epoch 00017: val_loss did not improve from 1.03113
Epoch 18/30
5304/5304 [==============================] - 0s 92us/step - loss: 1.0288 - acc: 0.4336 - val_loss: 1.0312 - val_acc: 0.4373

Epoch 00018: val_loss did not improve from 1.03113
Epoch 1

5299/5299 [==============================] - 0s 93us/step - loss: 1.0085 - acc: 0.4514 - val_loss: 1.0372 - val_acc: 0.4228

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.

Epoch 00024: val_loss did not improve from 1.03101
Epoch 25/30
5299/5299 [==============================] - 1s 104us/step - loss: 1.0084 - acc: 0.4595 - val_loss: 1.0362 - val_acc: 0.4278

Epoch 00025: val_loss did not improve from 1.03101
Epoch 26/30
5299/5299 [==============================] - 1s 95us/step - loss: 1.0081 - acc: 0.4518 - val_loss: 1.0373 - val_acc: 0.4244

Epoch 00026: val_loss did not improve from 1.03101
Epoch 27/30
5299/5299 [==============================] - 1s 96us/step - loss: 1.0103 - acc: 0.4497 - val_loss: 1.0376 - val_acc: 0.4228

Epoch 00027: val_loss did not improve from 1.03101
Epoch 28/30
5299/5299 [==============================] - 1s 95us/step - loss: 1.0060 - acc: 0.4535 - val_loss: 1.0414 - val_acc: 0.4228

Epoch 00028: val_loss did not improve from

5287/5287 [==============================] - 2s 349us/step - loss: 1.2078 - acc: 0.3770 - val_loss: 1.0183 - val_acc: 0.4541

Epoch 00001: val_loss improved from inf to 1.01827, saving model to models/forex_50_10.h5
Epoch 2/30
5287/5287 [==============================] - 1s 98us/step - loss: 1.0715 - acc: 0.4197 - val_loss: 1.0134 - val_acc: 0.4524

Epoch 00002: val_loss improved from 1.01827 to 1.01338, saving model to models/forex_50_10.h5
Epoch 3/30
5287/5287 [==============================] - 1s 95us/step - loss: 1.0383 - acc: 0.4303 - val_loss: 1.0135 - val_acc: 0.4524

Epoch 00003: val_loss did not improve from 1.01338
Epoch 4/30
5287/5287 [==============================] - 0s 93us/step - loss: 1.0339 - acc: 0.4371 - val_loss: 1.0140 - val_acc: 0.4354

Epoch 00004: val_loss did not improve from 1.01338
Epoch 5/30
5287/5287 [==============================] - 1s 106us/step - loss: 1.0273 - acc: 0.4309 - val_loss: 1.0120 - val_acc: 0.4337

Epoch 00005: val_loss improved from 1.01338

...

4548/4548 [==============================] - 3s 721us/step - loss: 0.8256 - acc: 0.5350 - val_loss: 0.8263 - val_acc: 0.5257

Epoch 00011: val_loss improved from 0.82691 to 0.82632, saving model to models/forex_80_114.h5
Epoch 12/30
4548/4548 [==============================] - 3s 729us/step - loss: 0.8203 - acc: 0.5165 - val_loss: 0.8280 - val_acc: 0.5138

Epoch 00012: val_loss did not improve from 0.82632
Epoch 13/30
4548/4548 [==============================] - 3s 720us/step - loss: 0.8204 - acc: 0.5211 - val_loss: 0.8282 - val_acc: 0.4901

Epoch 00013: val_loss did not improve from 0.82632
Epoch 14/30
4548/4548 [==============================] - 3s 721us/step - loss: 0.8196 - acc: 0.5264 - val_loss: 0.8255 - val_acc: 0.5257

Epoch 00014: val_loss improved from 0.82632 to 0.82547, saving model to models/forex_80_114.h5
Epoch 15/30
4548/4548 [==============================] - 3s 724us/step - loss: 0.8185 - acc: 0.5288 - val_loss: 0.8286 - val_acc: 0.5217

Epoch 00015: val_loss did not 

4542/4542 [==============================] - 3s 717us/step - loss: 0.8142 - acc: 0.5308 - val_loss: 0.8164 - val_acc: 0.5188

Epoch 00021: val_loss did not improve from 0.81193
Epoch 22/30
4542/4542 [==============================] - 3s 714us/step - loss: 0.8151 - acc: 0.5337 - val_loss: 0.8162 - val_acc: 0.5129

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.

Epoch 00022: val_loss did not improve from 0.81193
Epoch 23/30
4542/4542 [==============================] - 5s 1ms/step - loss: 0.8165 - acc: 0.5339 - val_loss: 0.8141 - val_acc: 0.5149

Epoch 00023: val_loss did not improve from 0.81193
Epoch 24/30
4542/4542 [==============================] - 3s 745us/step - loss: 0.8123 - acc: 0.5396 - val_loss: 0.8163 - val_acc: 0.5109

Epoch 00024: val_loss did not improve from 0.81193
Epoch 25/30
4542/4542 [==============================] - 3s 738us/step - loss: 0.8134 - acc: 0.5238 - val_loss: 0.8156 - val_acc: 0.5208

Epoch 00025: val_loss did not improve fr

500 windows sized
1000 windows sized
1500 windows sized
2000 windows sized
2500 windows sized
3000 windows sized
3500 windows sized
4000 windows sized
4500 windows sized
5000 windows sized
Tensor X/Y len:  5035  /  5035

Testing dataset dimension: [ 504 / 4531 ]

Train on 4531 samples, validate on 504 samples
Epoch 1/30
4531/4531 [==============================] - 334s 74ms/step - loss: 1.1153 - acc: 0.4339 - val_loss: 0.9128 - val_acc: 0.4464

Epoch 00001: val_loss improved from inf to 0.91278, saving model to models/forex_80_117.h5
Epoch 2/30
4531/4531 [==============================] - 6s 1ms/step - loss: 0.8822 - acc: 0.5003 - val_loss: 0.8658 - val_acc: 0.5198

Epoch 00002: val_loss improved from 0.91278 to 0.86575, saving model to models/forex_80_117.h5
Epoch 3/30
4531/4531 [==============================] - 3s 742us/step - loss: 0.8407 - acc: 0.4933 - val_loss: 0.8612 - val_acc: 0.5218

Epoch 00003: val_loss improved from 0.86575 to 0.86124, saving model to models/forex_80_117.h


Epoch 00008: val_loss did not improve from 0.81826
Epoch 9/30
4526/4526 [==============================] - 3s 725us/step - loss: 0.8239 - acc: 0.5247 - val_loss: 0.8190 - val_acc: 0.5348

Epoch 00009: val_loss did not improve from 0.81826
Epoch 10/30
4526/4526 [==============================] - 3s 734us/step - loss: 0.8157 - acc: 0.5223 - val_loss: 0.8197 - val_acc: 0.5129

Epoch 00010: val_loss did not improve from 0.81826
Epoch 11/30
4526/4526 [==============================] - 3s 756us/step - loss: 0.8160 - acc: 0.5245 - val_loss: 0.8193 - val_acc: 0.5646

Epoch 00011: val_loss did not improve from 0.81826
Epoch 12/30
4526/4526 [==============================] - 3s 726us/step - loss: 0.8181 - acc: 0.5179 - val_loss: 0.8240 - val_acc: 0.4871

Epoch 00012: val_loss did not improve from 0.81826
Epoch 13/30
4526/4526 [==============================] - 3s 722us/step - loss: 0.8147 - acc: 0.5347 - val_loss: 0.8204 - val_acc: 0.5229

Epoch 00013: val_loss did not improve from 0.81826
Epoc

4520/4520 [==============================] - 3s 729us/step - loss: 0.7988 - acc: 0.5420 - val_loss: 0.8505 - val_acc: 0.4831

Epoch 00019: val_loss did not improve from 0.84158
Epoch 20/30
4520/4520 [==============================] - 3s 741us/step - loss: 0.8006 - acc: 0.5347 - val_loss: 0.8540 - val_acc: 0.4712

Epoch 00020: val_loss did not improve from 0.84158
Epoch 21/30
4520/4520 [==============================] - 3s 729us/step - loss: 0.7950 - acc: 0.5423 - val_loss: 0.8475 - val_acc: 0.5249

Epoch 00021: val_loss did not improve from 0.84158
Epoch 22/30
4520/4520 [==============================] - 3s 750us/step - loss: 0.7915 - acc: 0.5409 - val_loss: 0.8474 - val_acc: 0.5169

Epoch 00022: val_loss did not improve from 0.84158
Epoch 23/30
4520/4520 [==============================] - 3s 728us/step - loss: 0.7996 - acc: 0.5274 - val_loss: 0.8447 - val_acc: 0.5209

Epoch 00023: val_loss did not improve from 0.84158
Epoch 24/30
4520/4520 [==============================] - 3s 723us/s